In [2]:
import requests
import pandas as pd
import numpy as np
import sddk
import zipfile
import io
import requests
from ithaca.util.alphabet import GreekAlphabet

In [3]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [4]:
PHI = s.read_file("SDAM_data/PHI/PHI_enriched_2020-12-16.json", "df")

In [5]:
# unfortunately, moving the dataset between Python and R causes that cells in most columns of the dataframe contain a LIST OF VALUES (of length 1) and not the VALUE itself.
# if this is the case, uncomment the last two rows to do one simple transformation

def lists_to_values(list_or_value):
  if isinstance(list_or_value, list):
    value = list_or_value[0]
  else: 
    value = list_or_value
  if not bool(value): # if is it empty dict or list
    value = np.nan
  return value
for column in PHI.columns:
  PHI[column] = PHI.apply(lambda row: lists_to_values(row[column]), axis=1)

In [6]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,NaN,12.0,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,NaN,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,NaN,13.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,NaN,56.0,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,NaN,6.0,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5


In [7]:
PHI[PHI["data"].apply(lambda x: isinstance(x, dict))]

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID


In [8]:
def dict_to_empty_str(x):
    if isinstance(x, dict):
        x = ""
    return x
PHI = PHI.applymap(dict_to_empty_str)
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,NaN,12.0,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,NaN,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,NaN,13.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,NaN,56.0,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,NaN,6.0,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5


# iphi

In [9]:
iphi = pd.read_json("../data/large_data/iphi.json")
len(iphi)

178551

In [10]:
len(PHI)

218162

In [11]:
PHI = PHI.merge(iphi.rename(columns={"id":"PHI_ID"}), on="PHI_ID", how="outer")

In [12]:
PHI

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata_x,data,...,text,metadata_y,region_main_id,region_main,region_sub_id,region_sub,date_str,date_min,date_max,date_circa
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,NaN,12.0,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,...,εδοχσεν τοι δεμοι τ[ος ε σ]αλαμ[ινι κλεροχ]ος ...,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,1701.0,Attica (IG I-III),1700.0,Attica,c. 510-500 a.,-510.0,-500.0,0.0
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,NaN,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,...,[--------]ν βολ ---------- [------] α εκον ---...,Att. — non-stoich. — c. 500 a.,1701.0,Attica (IG I-III),1700.0,Attica,c. 500 a.,-500.0,-500.0,0.0
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,NaN,13.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,...,[-]αρ[----]ι ερακλειο[-----] [-]αρο [-] τιθενα...,Att. — stoich. 21 — 490-480 a.,1701.0,Attica (IG I-III),1700.0,Attica,490-480 a.,-490.0,-480.0,0.0
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,NaN,56.0,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,...,[--------------------------------------] [----...,Att. — stoich. 38 — 485/4 a.,1701.0,Attica (IG I-III),1700.0,Attica,485/4 a.,-485.0,-484.0,0.0
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,NaN,6.0,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,...,[εδοχσε]ν [ τει βολει] και [τ]οι δεμοι οτε παρ...,Att. — c. 500 a.,1701.0,Attica (IG I-III),1700.0,Attica,c. 500 a.,-500.0,-500.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218161,/text/353790?location=1682&patt=&bookid=884&bo...,IK Strat. III,Add. et Corr. 1224a,Regions\nAsia Minor\nCaria,IK Strat. III\nAdd. et Corr. 1224a,Caria — Stratonikeia — Roman period — IK Strat...,Epitaph of Aelius (?) Florentinus. Marble ped...,2.0,2\n,"Aẹḷ(ium?) Florentinum, opt(ionem)\nvexillati...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[--- τετικ]μενος ειτου.,"Phryg. — Holmoi?, W. of: Inli, Kali Çay — ZVS ...",1702.0,Asia Minor,1679.0,Phrygia,,NaN,NaN,NaN
218163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,αυρ. κυριλλα μυρωνος συμβιος αυρ. παπα μενεου ...,Phryg. — Amorion (Hisarköy) — Piribeyli — ZVS ...,1702.0,Asia Minor,1679.0,Phrygia,,NaN,NaN,NaN
218164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[α]υρ. τειμοθεος γλυκυτατω συνβιω ιταλικη ανεσ...,"Phryg., E. — Amorion (Hisarköy) — Aşaǧa Piribe...",1702.0,Asia Minor,1679.0,Phrygia,3rd c. AD,201.0,300.0,0.0


In [14]:
url = "https://raw.githubusercontent.com/sommerschield/iphi/main/train/data/iphi_text_clean.py"
exec(requests.get(url).text)

In [25]:
import re
import unicodedata

import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


def strip_accents(s):
    '''Strips all accents.'''
    return ''.join(c for c in unicodedata.normalize('NFD', s) if
                   unicodedata.category(c) != 'Mn')


def text_to_sentences(t, alphabet):
    '''Tokenizes sentences and removes the empty ones.'''
    sentences = []
    for s in nltk.tokenize.sent_tokenize(t):
        # remove all puntuation from sentence
        s = re.sub(r'[%s]+' % ''.join(alphabet.punctuation), ' ', s)

        # collapse spaces
        s = re.sub(r'\s+', ' ', s).strip()

        # append sentence
        if len(s) > 1:
            sentences.append(s)
    return sentences


def text_clean_phi(text_cleaned, alphabet):
    '''Converts an inscription text to a machine-actionable format.'''

    # Remove lines that start with
    text_cleaned = re.sub(r'^(IG|SEG|BCH|Agora|vacat) .*\n?',
                          '', text_cleaned, flags=re.MULTILINE)

    # Replace double brackets with single
    text_cleaned = text_cleaned.replace('〚', '[').replace('〛', ']')

    # Remove everything after vacat
    text_cleaned = re.sub(r'vacat .*\n?', '\n', text_cleaned, flags=re.MULTILINE)

    # Split on :
    text_cleaned = re.sub(r' [:∶]+ ', '. ', text_cleaned)

    # Join lines ending with -
    text_cleaned = re.sub(r'-\n', r'', text_cleaned)

    # Remove Greek numerals with ʹ
    text_cleaned = re.sub(r'[\w∙]+ʹ', r'', text_cleaned)

    # Remove Greek numerals
    word_boundary = r'([\s\.\⏑\—\-\-\,⏑․\[\]]|$|^)'
    greek_numerals = re.escape(
        '∶ΠT𐅈𐅃𐅉ϛ𐅀𐅁𐅂Ι𐅃ΔͰΗΧΜΤ𐅄𐅅𐅆𐅇𐅈𐅉𐅊𐅋𐅌𐅍𐅎𐅏𐅏𐅐𐅑��𐅓𐅔𐅕𐅖')
    text_cleaned = re.sub(rf'\[[{greek_numerals}]+\]', '', text_cleaned)
    text_cleaned = re.sub(
        rf'{word_boundary}([{greek_numerals}]+){word_boundary}',
        lambda m: '%s0%s' % (m.group(1), m.group(3)), text_cleaned)
    text_cleaned = re.sub(
        rf'{word_boundary}([{greek_numerals}]+){word_boundary}',
        lambda m: '%s0%s' % (m.group(1), m.group(3)), text_cleaned)

    # Remove extra punctuation
    text_cleaned = re.sub(r'(\s*)[\∶|\⋮|\·|\⁙|\;]+(\s*)', r' ', text_cleaned)

    # Remove all (?)
    text_cleaned = re.sub(r'\s*\(\?\)', r'', text_cleaned)

    # Remove anything between {}
    text_cleaned = re.sub(r'{[^}]*}', r'', text_cleaned)

    # Remove < >, but keep content
    text_cleaned = re.sub(r'<([^>]*)>', r'\1', text_cleaned)

    # Remove latin numbering within brackets [I]
    text_cleaned = re.sub(
        r'\[M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\]\s*$', r'',
        text_cleaned)

    # Lowercase
    text_cleaned = text_cleaned.lower()

    # filter alphabet to replace tonos and h
    text_cleaned = alphabet.filter(text_cleaned)

    # Remove vac, v v. vac vac. vac.? in (), etc
    text_cleaned = re.sub(r'(\d+\s)?\s*v[\w\.\?]*(\s\d+(\.\d+)?)?', '',
                          text_cleaned)

    # Remove parentheses surrounding greek characters
    text_cleaned = re.sub(r'\(([{}]+)\)'.format(''.join(alphabet.alphabet)),
                          r'\1', text_cleaned)

    # Remove any parentheses that has content that is not within the greek alphabet
    text_cleaned = re.sub(r'\([^\)]*\)', r'', text_cleaned)

    # Replace c with σ
    text_cleaned = text_cleaned.replace('ϲ', 'σ')

    # Remove lines with >10% Latin characters
    text_cleaned = '\n'.join([line for line in text_cleaned.splitlines() if len(
        line) and len(re.findall(r'[a-z]', line)) / len(line) < 0.1])

    # Convert short syllables to long ones
    text_cleaned = text_cleaned.replace('⏑', '—')
    text_cleaned = text_cleaned.replace('⏕', '—')
    text_cleaned = text_cleaned.replace('-', '—')

    # Collapse space between dashes (on purpuse double)
    text_cleaned = re.sub(r'—(?:[\s]+—)+',
                          lambda g: re.sub(r'[\s]+', '', g.group(0)),
                          text_cleaned, flags=re.MULTILINE)

    # Replace dots with dashes and c.#
    text_cleaned = re.sub(
        r"(?:\.|․|—)+\s?(?:c\.)?(\d+)(?:(\-|-|—)\d+)?\s?(?:\.|․|—)*",
        lambda g: alphabet.missing * int(g.group(1)),
        text_cleaned, flags=re.MULTILINE)

    # Replace with missing character
    text_cleaned = text_cleaned.replace(u'\u2013', alphabet.missing)
    text_cleaned = text_cleaned.replace(u'\u2014', alphabet.missing)
    text_cleaned = text_cleaned.replace('․', alphabet.missing)

    # PHI #⁷removed automatically because only gr chars

    # Join ][ and []
    text_cleaned = text_cleaned.replace('][', '').replace('[]', '')

    # Keep only alphabet characters
    chars = re.escape(''.join(
        alphabet.alphabet + alphabet.numerals + alphabet.punctuation + [
            alphabet.space, alphabet.missing, alphabet.sog, alphabet.eog]))
    #text_cleaned = re.sub(rf'[^{chars}]', ' ', text_cleaned)

    # Replace any digit with 0
    text_cleaned = re.sub(r'\d+', '0', text_cleaned)

    # Merge multiple 0
    text_cleaned = re.sub(r'(\s+0)+', r' 0', text_cleaned)

    # Remove []
    text_cleaned = re.sub(r'\[\s*\]', '', text_cleaned)

    # Remove space before punctuation / merge double punctuation
    chars = re.escape(''.join(alphabet.punctuation + [alphabet.eog]))
    text_cleaned = re.sub(rf'\s+([{chars}])', r'\1', text_cleaned)

    # Remove leading space and punctuation
    text_cleaned = text_cleaned.lstrip(
        ''.join(alphabet.punctuation + [alphabet.space]))

    # Collapse duplicate dots
    punc = re.escape(''.join(alphabet.punctuation))
    text_cleaned = re.sub(rf'([{punc}])+', r'\1', text_cleaned)

    # Collapse spaces
    text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()

    return text_cleaned

In [26]:
alphabet = GreekAlphabet()

In [27]:
test_text = PHI["data"].tolist()[0]
test_text

'ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ]ος\nοἰκε͂ν ἐᾶ Σαλαμῖνι [․․5․․]λεν [․․․7․․․ Ἀθέ]νε-\nσι τελε͂ν καὶ στρατ[εύεσθ]αι ⋮ τ̣[ὰ δ’ ἐ Σαλαμῖνι] μ-\nὲ μι[σθ]ο͂ν, ἐὰ μὲ οἰκ[․․․7․․․]ο[․ μισθόμενο․ ⋮ ἐ]ὰ-\nν δὲ μισθο͂ι, ἀποτί[νεν τὸ μισθόμενον καὶ τὸ] μ̣-\nισθο͂ντα ℎεκάτε[ρον ․․․․․․․․19․․․․․․․․․]\nἐς δεμόσιο[ν ∶ ἐσπράτεν δὲ τὸν ἄ]-\nρχο[ν]τα, ἐὰν [δὲ μέ, εὐθ]ύ[νεσθαι ∶ τ]-\nὰ δὲ [ℎ]όπλα π[αρέχεσ]θ̣α[ι αὐτὸς ∶ τ]-\nριά[κ]οντα ∶ δρ[αχμο͂ν ⋮] ℎο[πλισμένο]-\nν δὲ [τ]ὸν ἄρχοντ[α τὰ ℎόπλα κρίν]-\nεν ⋮ [ἐπ]ὶ τε͂ς β[ο]λε͂[ς ․․․c.11․․․․]'

In [23]:
PHI["text"].tolist()[0]

'εδοχσεν τοι δεμοι τ[ος ε σ]αλαμ[ινι κλεροχ]ος οικεν εα σαλαμινι [-----]λεν [------- αθε]νεσι τελεν και στρατ[ευεσθ]αι τ[α δ ε σαλαμινι] με μι[σθ]ον εα με οικ[-------]ο[- μισθομενο- ε]αν δε μισθοι αποτι[νεν το μισθομενον και το] μισθοντα εκατε[ρον -------------------] ες δεμοσιο[ν εσπρατεν δε τον α]ρχο[ν]τα εαν [δε με ευθ]υ[νεσθαι. τ]α δε [ο]πλα π[αρεχεσ]θα[ι αυτος. τ]ρια[κ]οντα. δρ[αχμον] ο[πλισμενο]ν δε [τ]ον αρχοντ[α τα οπλα κριν]εν [επ]ι τες β[ο]λε[ς -----------].'

In [28]:
text_clean_phi(test_text, alphabet)

'ἔδοχσεν τοι δέμοι τ[ὸς ἐ σ]αλαμ[ῖνι κλερόχ]ος οἰκεν ἐᾶ σαλαμῖνι [-----]λεν [------- ἀθέ]νεσι τελεν καὶ στρατ[εύεσθ]αι τ[ὰ δ’ ἐ σαλαμῖνι] μὲ μι[σθ]ον, ἐὰ μὲ οἰκ[-------]ο[- μισθόμενο- ἐ]ὰν δὲ μισθοι, ἀποτί[νεν τὸ μισθόμενον καὶ τὸ] μισθοντα ἑκάτε[ρον -------------------] ἐς δεμόσιο[ν. ἐσπράτεν δὲ τὸν ἄ]ρχο[ν]τα, ἐὰν [δὲ μέ, εὐθ]ύ[νεσθαι. τ]ὰ δὲ [ὅ]πλα π[αρέχεσ]θα[ι αὐτὸς. τ]ριά[κ]οντα. δρ[αχμον] ὁ[πλισμένο]ν δὲ [τ]ὸν ἄρχοντ[α τὰ ὅπλα κρίν]εν [ἐπ]ὶ τες β[ο]λε[ς -----------]'

# Pythia data

In [30]:
url = "SDAM_data/PHI/PHI_pythia/phi-plaintext.zip"
resp = conf[0].get(conf[1] + url)

In [31]:
zipped = zipfile.ZipFile(io.BytesIO(resp.content))

In [32]:
namelist = zipped.namelist()
namelist = [file for file in namelist if (".txt" in file) and (file[0] != "_")]
namelist[:10]

['phi-plaintext/285586.txt',
 'phi-plaintext/186989.txt',
 'phi-plaintext/180320.txt',
 'phi-plaintext/284840.txt',
 'phi-plaintext/261172.txt',
 'phi-plaintext/3644.txt',
 'phi-plaintext/249826.txt',
 'phi-plaintext/152266.txt',
 'phi-plaintext/239319.txt',
 'phi-plaintext/5235.txt']

In [50]:
len(namelist)

42711

In [33]:
### to do below some matching, we have to take care of appropriate encoding
from unicodedata import normalize
def normalize_string(string):
    try:
        return normalize("NFC", string)
    except: 
        return ""

In [34]:
phi_pythia_dict = {}
for filename in namelist:
    phi_number = filename.rpartition("/")[2].partition(".txt")[0]
    phi_string = normalize_string(zipped.read(filename).decode())
    phi_pythia_dict[phi_number] =  phi_string

In [35]:
phi_pythia_dict["50"]

'------------------------------- ερε[------------------------------------------------- ἱππ]όνικο[ς εἶπε ----------------------------------------------] ἑκαστο[---------------------------------- μισθὸν λαμβάνειν δ]ραχμὲν τε[ς ἑμέρας ἑκάστες ἐπιμέλεσθαι δὲ αὐτὸς καὶ τες κρένες κα]ὶ τες ἀγογες [το ὕδατος ----------------------- νικόμαχος εἶπε τὰ] μὲν ἄλλα καθ[άπερ τει βολει ---------------------------------] ει ὅπος ἂν ῥέοσ[ι ------------------------------ ὅπος δ ἂν ἀπὸ ὀ]λιγίστον χρεμάτο[ν --------------------- τὸς πρυτάνες ὁὶ ἂν λάχ]οσι προτοι πρυτανεύ[εν ----------------------- ἐν τει πρότει τον] κυρίον ἐκκλεσιον προτ[ον μετὰ τὰ ἱερὰ --------------------- ἀγα]θὸν ε ναι τοι δέμοι τοι ἀθε[ναίον ---------------------------- με]δεμία γίγνεται καὶ ἔχει ἀθε[ναίοις ὁς ----------------------- ε]ἶπε τὰ μὲν ἄλλα καθάπερ νικόμα[χος ἐπαινέσαι δὲ καὶ περικλεῖ καὶ παρ]άλοι καὶ χσανθίπποι καὶ τοῖς ὑέ[σιν ἀπαναλίσκεν δὲ ἀπὸ τον χρεμάτον] ὅσα ἐς τὸν φόρον τὸν ἀθεναίον τελ[εται ἐπειδὰν ἑ θεὸς ἐχς αὐτον 

In [36]:
def get_pythia_text_by_number(phi_number):
    try: return phi_pythia_dict[str(phi_number)]
    except: return ""

In [37]:
PHI["string_pythia"] = PHI["PHI_ID"].apply(get_pythia_text_by_number)  

In [38]:
PHI.head(10)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,string_pythia
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,,12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1,ἔδοχσεν τοι δέμοι τ[ὸς ἐ σ]αλαμ[ῖνι κλερόχ]ος ...
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,,14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2,[--------9---]ν βολ ---------- [------7--] α ἑ...
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,,13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3,[-]αρ[----]ι ἑρακλειο[-----] [-]αρο [-] τιθένα...
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,,56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4,[--------------------------------------] [----...
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,,6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5,[ἔδοχσε]ν [ τει βολει] καὶ [τ]οι δέμοι ὅτε παρ...
5,/text/6?location=1701&patt=&bookid=4&offset=0&...,IG I³,6,Regions\nAttica (IG I-III),IG I³\n6,Att. — stoich. 23/11 — ante 460 a.,,160,face A.BM 309.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n...,— — — — — — — — — — — — —\n[․․․․․․15․․․․․․․] δ...,IG-I³.csv,6,------------- [---------------] δραχμεισ[ι ---...
6,/text/7?location=1701&patt=&bookid=4&offset=0&...,IG I³,7,Regions\nAttica (IG I-III),IG I³\n7,Att. — stoich. 40 — 460-450,,28,frg. a.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n13\n\n\nf...,[ἔδοχσεν τε͂]ι βο[λ]ε͂[ι καὶ το͂ι δέμοι· ․․6․․...,IG-I³.csv,7,[ἔδοχσεν τε]ι βο[λ]ε[ι καὶ τοι δέμοι ------ ἐπ...
7,/text/8?location=1701&patt=&bookid=4&offset=0&...,IG I³,8,Regions\nAttica (IG I-III),IG I³\n8,Att. — stoich. 32 — 460-450,,26,frg. a.1\n\n\n\n5\n\n\n\n\n10\n\n\nfrg. b.12\n...,[․․5․․]#⁷ον ℎὰ ο[․․․․․․․․․21․․․․․․․․․․]\nα περ...,IG-I³.csv,8,[-----] ον ἃ ο[---------------------] α περὶ τ...
8,/text/9?location=1701&patt=&bookid=4&offset=0&...,IG I³,9,Regions\nAttica (IG I-III),IG I³\n9,Att. — stoich. 24 — c. 458 a.,,17,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n,[ἔδοχσεν τε͂ι βο]λε̣͂ι καὶ το͂[ι δέμ]-\n[οι· ․...,IG-I³.csv,9,[ἔδοχσεν τει βο]λει καὶ το[ι δέμοι ---ντὶς ἐπρ...
9,/text/10?location=1701&patt=&bookid=4&offset=0...,IG I³,10,Regions\nAttica (IG I-III),IG I³\n10,Att. — stoich. 22 — 469-450,,28,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,[ἔδο]ξεν τῆι βολῆι καὶ τῶι δ[ή]-\n[μωι· Ἀ]καμα...,IG-I³.csv,10,[ἔδο]ξεν τῆι βολῆι καὶ τῶι δ[ήμωι ἀ]καμαντὶς [...


In [42]:
len(PHI[PHI["string_pythia"] != ""])

42710

In [44]:
def extend_pythia(row):
    if row["string_pythia"] == "":
        return row["data"]
    else: 
        return row["string_pythia"]

In [45]:
PHI["string_pythia"] = PHI.apply(lambda row: extend_pythia(row), axis=1)

In [51]:
PHI["data"].apply(lambda x: len(x.split())).sum()

6998420

In [52]:
PHI["string_pythia"].apply(lambda x: len(x.split())).sum()

5412718

In [49]:
sddk.write_file("SDAM_data/PHI/PHI_with_pythia_20201216.json", PHI, conf)

A file with the same name ("PHI_with_pythia_20201216.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_with_pythia_20201216.json"
